In [1]:
from sklearn.utils.fixes import sklearn
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import seaborn as sn
import sklearn.preprocessing as preprosessing

# 数据准备
features = pd.read_csv('d:/GitRepos/data/temps.csv')
features = pd.get_dummies(features)
labels = np.array(features['actual'])
features = features.drop(['year','actual'],axis=1)
features = np.array(features)
features = preprosessing.StandardScaler().fit_transform(features)
features.shape
# labels.shape


(348, 13)

In [2]:
# 网络搭建
in_features = 13
hidden_features = 128
out_features = 1
batch_size = 16
mnn = nn.Sequential(
    nn.Linear(in_features,hidden_features),
    nn.Sigmoid(),
    nn.Linear(hidden_features,out_features)
)
cost = nn.MSELoss()
optimizer = torch.optim.Adam(mnn.parameters(),lr = 0.001)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# mnn.to(device) 
losses = []
# 训练
for epoch in range(1000):
    batch_loss = []
    for start in range(0,len(features),batch_size):
        end = start + batch_size if (start + batch_size < len(features)) else len(features)
        xx = torch.tensor(features[start:end],dtype=torch.float,requires_grad=True)
        yy = torch.tensor(labels[start:end],dtype=torch.float,requires_grad=True)
        predict = mnn(xx)
        loss = cost(predict,yy)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        batch_loss.append(loss.data.numpy())
    
    if epoch % 100 == 0:
        losses.append(batch_loss)
        print("epoch {0},loss {1}".format(epoch,np.mean(batch_loss)))

e:\Program2\Anaconda3\envs\pytorch\lib\site-packages\torch\nn\modules\loss.py:520: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
e:\Program2\Anaconda3\envs\pytorch\lib\site-packages\torch\nn\modules\loss.py:520: UserWarning: Using a target size (torch.Size([12])) that is different to the input size (torch.Size([12, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


epoch 0,loss 3862.596923828125
epoch 100,loss 37.939170837402344
epoch 200,loss 35.660274505615234
epoch 300,loss 35.2928466796875
epoch 400,loss 35.11722183227539
epoch 500,loss 34.9793586730957
epoch 600,loss 34.85498809814453
